In [1]:
import BAC0

In [2]:
bacnet = BAC0.connect(port=47811)

2024-12-23 12:40:32,664 - INFO    | Starting BAC0 version 23.07.03 (Lite)
2024-12-23 12:40:32,665 - INFO    | Use BAC0.log_level to adjust verbosity of the app.
2024-12-23 12:40:32,665 - INFO    | Ex. BAC0.log_level('silence') or BAC0.log_level('error')
2024-12-23 12:40:32,666 - INFO    | Starting TaskManager
2024-12-23 12:40:32,666 - INFO    | Using ip : 10.84.0.66:47811 on port 47811 | broadcast : 10.84.0.255
2024-12-23 12:40:32,701 - INFO    | Starting app...
2024-12-23 12:40:32,702 - INFO    | BAC0 started
2024-12-23 12:40:32,702 - INFO    | Registered as Simple BACnet/IP App
2024-12-23 12:40:32,703 - INFO    | Device instance (id) : 3056327
2024-12-23 12:40:32,724 - INFO    | Update Local COV Task started (required to support COV)


In [5]:
def leer_punto(ip,tipo,direccion):
    try:
        return bacnet.read(f'{ip} {tipo} {direccion} presentValue')
    except:
        return ''
"""
def descubrir_puntos(ip):
    tipos = ["binaryOutput",
    "binaryInput",
    "multiStateOutput",
    "multiStateInput",
    "analogValue",
    "analogInput",
    "analogOutput"
    ]
    for direccion in range(0,)
    """
        

'\ndef descubrir_puntos(ip):\n    tipos = ["binaryOutput",\n    "binaryInput",\n    "multiStateOutput",\n    "multiStateInput",\n    "analogValue",\n    "analogInput",\n    "analogOutput"\n    ]\n    for direccion in range(0,)\n    '

In [10]:
leer_punto("10.84.67.185", "binaryOutput", "513")

'active'

In [44]:
tabla = bacnet.readMultiple("10.84.67.185 device 1001 all")

In [45]:
puntos = tabla[-15]
tabla[-15]

[('device', 9001),
 ('binaryOutput', 257),
 ('binaryInput', 258),
 ('binaryOutput', 259),
 ('binaryInput', 260),
 ('binaryInput', 261),
 ('binaryValue', 262),
 ('multiStateOutput', 263),
 ('multiStateInput', 264),
 ('binaryOutput', 265),
 ('binaryInput', 266),
 ('multiStateOutput', 267),
 ('multiStateInput', 268),
 ('analogValue', 269),
 ('analogInput', 270),
 ('binaryInput', 271),
 ('analogInput', 272),
 ('multiStateOutput', 273),
 ('multiStateInput', 274),
 ('analogInput', 275),
 ('analogInput', 276),
 ('multiStateOutput', 277),
 ('multiStateInput', 278),
 ('binaryOutput', 279),
 ('binaryInput', 280),
 ('binaryOutput', 281),
 ('binaryInput', 282),
 ('analogValue', 283),
 ('analogValue', 284),
 ('analogInput', 285),
 ('analogInput', 286),
 ('binaryOutput', 287),
 ('binaryInput', 288),
 ('binaryOutput', 289),
 ('binaryInput', 290),
 ('binaryOutput', 291),
 ('binaryInput', 292),
 ('analogValue', 293),
 ('analogInput', 294),
 ('analogValue', 295),
 ('analogInput', 296),
 ('analogValue', 

In [1]:
import json
mapeo_modbus = []
coil_offset = 0
register_offset = 0

for tipo, direccion in puntos:
    if tipo in ["binaryInput", "binaryOutput"]:
        mapeo_modbus.append({
            "bacnet_tipo": tipo,
            "bacnet_direccion": direccion,
            "modbus_tipo": "Coil",
            "modbus_direccion": coil_offset,
        })
        coil_offset += 1
    elif tipo in ["analogInput", "analogOutput", "analogValue"]:
        mapeo_modbus.append({
            "bacnet_tipo": tipo,
            "bacnet_direccion": direccion,
            "modbus_tipo": "Holding Register",
            "modbus_direccion": register_offset,
        })
        register_offset += 1

with open("archivo.json", "w") as archivo:
    json.dump(mapeo_modbus, archivo, indent=4)

In [39]:
from pymodbus.client import ModbusTcpClient

# Configuración del cliente
client = ModbusTcpClient('10.84.0.66', port=502)

# Conexión
if client.connect():
    # Lectura de 10 registros de retención desde la dirección 0
    response = client.read_holding_registers(address=0, count=10, unit=1)

    if not response.isError():  # Verifica si no hubo errores
        print("Registros:", response.registers)
    else:
        print("Error al leer registros:", response)

    # Cerrar conexión
    client.close()
else:
    print("No se pudo conectar al dispositivo Modbus.")


Connection to (10.84.0.66, 502) failed: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión


No se pudo conectar al dispositivo Modbus.
